In [1]:
import ipywidgets as widgets
import time
import asyncio

In [2]:
from IPython.display import display

In [3]:
from pynq.overlays.arm_pynq import arm_pynqOverlay

In [4]:
import pynq.lib as lib
import cffi
_ffi = cffi.FFI()

In [5]:
overlay = arm_pynqOverlay('arm_pynq.bit')

In [6]:
iic  = lib.AxiIIC(overlay.ip_dict['axi_iic_0'])

In [7]:
tx_buf = _ffi.new("unsigned char [32]")
rx_buf = _ffi.new("unsigned char [32]")

In [8]:
tx_buf[0] = 0xfe
tx_buf[1] = 0x64
iic.send(0x40,tx_buf,2,0)

2

In [9]:
tx_buf[0] = 0x00
tx_buf[1] = 0x80
iic.send(0x40,tx_buf,2,0)

2

In [10]:
tx_buf[0] = 0x00
iic.send(0x40,tx_buf,1,1)
iic.receive(0x40,rx_buf,1,0)


1

In [11]:
rx_buf[0]

0

In [12]:
tx_buf[0] = 0xfe
iic.send(0x40,tx_buf,1,1)
iic.receive(0x40,rx_buf,1,0)


1

In [13]:
rx_buf[0]

100

In [14]:
#set up the base counts 
tx_buf[0] = 0x06
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)
tx_buf[0] = 0x07
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)

tx_buf[0] = 0x0a
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x0b
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#upper

tx_buf[0] = 0x0e
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x0f
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#upper

tx_buf[0] = 0x12
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x13
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#upper

tx_buf[0] = 0x16
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x17
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#upper

tx_buf[0] = 0x1a
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x1b
tx_buf[1] = 0x00
iic.send(0x40,tx_buf,2,0)#upper

2

In [15]:
def close_grasp(b):
    tx_buf[0] = 0x18
    tx_buf[1] = 0x00
    iic.send(0x40,tx_buf,2,0)#low
    tx_buf[0] = 0x19
    tx_buf[1] = 0x08
    iic.send(0x40,tx_buf,2,0)#upper

In [16]:
def close_open(b):
    tx_buf[0] = 0x18
    tx_buf[1] = 0x01
    iic.send(0x40,tx_buf,2,0)#low
    tx_buf[0] = 0x19
    tx_buf[1] = 0x00
    iic.send(0x40,tx_buf,2,0)#upper

In [17]:
GraspClose = widgets.Button(description="Grasp Close")
GraspOpen = widgets.Button(description="Grasp Open")

In [18]:
inital_pos = 369
#forward back        
tx_buf[0] = 0x08
tx_buf[1] = inital_pos & 0xff
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x09
tx_buf[1] = inital_pos >>8
iic.send(0x40,tx_buf,2,0)#upper

#updown        
tx_buf[0] = 0x0c
tx_buf[1] = inital_pos & 0xff
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x0d
tx_buf[1] = inital_pos >>8
iic.send(0x40,tx_buf,2,0)#upper

#wrist tilt
tx_buf[0] = 0x14
tx_buf[1] = inital_pos & 0xff
iic.send(0x40,tx_buf,2,0)#low
tx_buf[0] = 0x15
tx_buf[1] = inital_pos >>8
iic.send(0x40,tx_buf,2,0)#upper

2

In [19]:
%gui asyncio

In [20]:
WristTilt = widgets.IntSlider(value=369,description="Wrist Tilt", min=122,max=616,orientation='vertical')
UpDwn = widgets.IntSlider(value=369,description="Up Down",min=122,max=616,orientation='vertical')
fwdbck = widgets.IntSlider(value=369,description="Forward Back",min=122,max=616,)
rot = widgets.IntSlider(value=369,description="Rotate",min=122,max=616,)
out = widgets.Output()

In [21]:
def wait_for_change(widget,  value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [22]:
async def rotate():
        # out.append_stdout('did work ' + '\n')
    while True:
        #rot
        rot_pos = await wait_for_change(rot, 'value')
        tx_buf[0] = 0x1c
        tx_buf[1] = rot_pos & 0xff
        iic.send(0x40,tx_buf,2,0)#low
        tx_buf[0] = 0x1d
        tx_buf[1] = rot_pos >>8
        #out.append_stdout('async rot ' + str(rot_pos) + '\n')
        iic.send(0x40,tx_buf,2,0)#upper       
asyncio.ensure_future(rotate()) 

<Task pending coro=<rotate() running at <ipython-input-22-aa31f3e19a7a>:1>>

In [23]:
async def fb():
        # out.append_stdout('did work ' + '\n')
    while True:               
        #fwd back
        fb_pos = await wait_for_change(fwdbck, 'value')
        #out.append_stdout('async forward' + str(fb_pos) + '\n')
        tx_buf[0] = 0x08
        tx_buf[1] = fb_pos & 0xff
        iic.send(0x40,tx_buf,2,0)#low
        tx_buf[0] = 0x09
        tx_buf[1] = fb_pos >>8
        iic.send(0x40,tx_buf,2,0)#upper  
asyncio.ensure_future(fb()) 

<Task pending coro=<fb() running at <ipython-input-23-3fcd68a6fd24>:1>>

In [24]:
async def wrist():
        # out.append_stdout('did work ' + '\n')
    while True:               
            
        #wrist tilt
        Wrist_tilt = await wait_for_change(WristTilt, 'value')
        #out.append_stdout('async wrist' + str(Wrist_tilt) + '\n')
        tx_buf[0] = 0x14
        tx_buf[1] = Wrist_tilt & 0xff
        iic.send(0x40,tx_buf,2,0)#low
        tx_buf[0] = 0x15
        tx_buf[1] = Wrist_tilt >>8
        iic.send(0x40,tx_buf,2,0)#upper
asyncio.ensure_future(wrist()) 

<Task pending coro=<wrist() running at <ipython-input-24-b0f07d4f2d55>:1>>

In [25]:
async def updown():
    while True:               
        
        #updown
        UpDwn_pos = await wait_for_change(UpDwn, 'value')
        #out.append_stdout('async updown' + str(UpDwn_pos) + '\n')
        tx_buf[0] = 0x0c
        tx_buf[1] = UpDwn_pos & 0xff
        iic.send(0x40,tx_buf,2,0)#low
        tx_buf[0] = 0x0d
        tx_buf[1] = UpDwn_pos >>8
        iic.send(0x40,tx_buf,2,0)#upper
asyncio.ensure_future(updown()) 

<Task pending coro=<updown() running at <ipython-input-25-19cf9c1679c7>:1>>

In [43]:
#display(out)
#display(WristTilt)
#display(UpDwn)
#display(fwdbck)
#display(rot)



Button(description='Grasp Close', style=ButtonStyle())

Button(description='Grasp Open', style=ButtonStyle())

In [26]:
from ipywidgets import Layout, HBox, VBox, Label

In [27]:
#left_box = VBox([fwdbck,rot])
#right_box = VBox([WristTilt,UpDwn ])
HBox([UpDwn,WristTilt,rot,fwdbck])


In [28]:
display(GraspClose)
display(GraspOpen)
GraspClose.on_click(close_grasp)
GraspOpen.on_click(close_open)

Button(description='Grasp Close', style=ButtonStyle())

Button(description='Grasp Open', style=ButtonStyle())